# Perform MAST Differential Expression Tests

In this notebook, we retrieve our CD4 and CD8 T cells and our cell type labels, then perform MAST differential gene expression tests. Comparisons will be carried out between Pediatric (UP1 cohort) and Older Adult (BR2 cohort).

To balance cell counts, we'll group cells by sample, then use the minimum number of cells across all samples. For example, to test CD4 Naive T cells , we'll examine the number of CD4 Naive cells in each subject and randomly sample based on the minimum counts from all 8 subjects across all samples. After sampling from each sample, we'll then compare the cells in the two cohorts using MAST.

For MAST, we need to also include Cellular Detection Rate (CDR) as a cofactor to control for gene expression differences between samples.

## Load packages

hise: The Human Immune System Explorer R SDK package  
purrr: Functional programming tools  
furrr: Parallelization of functional programming using `futures`  
dplyr: Dataframe handling functions  
ggplot2: plotting functions  
Seurat: single cell genomics methods  
MAST: single cell differential expression tests

In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(hise)
quiet_library(purrr)
quiet_library(furrr)
quiet_library(dplyr)
quiet_library(ggplot2)
quiet_library(Seurat)
quiet_library(MAST)

In [2]:
read_path_uuid <- function(uuid) {
    uuid_path <- paste0("cache/", uuid)
    if(!dir.exists(uuid_path)) {
        cacheFiles(list(uuid))
    }
    list.files(uuid_path, full.names = TRUE)[1]
}

In [3]:
read_csv_uuid <- function(uuid) {
    filename <- read_path_uuid(uuid)
    read.csv(filename)
}

## Retrieve files

Now, we'll use the HISE SDK package to retrieve the Seurat objects and cell type labels based on file UUIDs. This will be placed in the `cache/` subdirectory by default.

In [4]:
meta_uuid <- "5e3115d4-9207-4020-8e3a-3792dd28ea6b"
sample_meta <- read_csv_uuid(meta_uuid)

In [29]:
file_uuids <- list(
    cd4_so = "50756d6d-8216-4eba-8245-0a41d4f44e7a", # CD4 T cell Seurat object
    cd8_so = "057a9e09-1f9d-4684-b00f-7739209ffe17", # CD8 T cell Seurat object
    cd4_labels = "56da30a4-3ac1-4c2b-8ca8-5e1d58cc6986", # CD4 type labels
    cd8_labels = "d369cedf-18c3-4579-83ea-a0daab116e3c"  # CD8 type labels
)

In [30]:
file_paths <- map(file_uuids, read_path_uuid)

## Select cells


In [31]:
cd4_labels <- read.csv(file_paths$cd4_labels)
cd8_labels <- read.csv(file_paths$cd8_labels)

In [8]:
all_labels <- rbind(cd4_labels, cd8_labels)

In [9]:
all_labels <- left_join(all_labels, sample_meta)

Joining with `by = join_by(sample.sampleKitGuid)`


In [10]:
head(all_labels)

,barcodes,sample.sampleKitGuid,predicted.celltype.l1.score,predicted.celltype.l1,predicted.celltype.l2.score,predicted.celltype.l2,predicted.celltype.l3.score,predicted.celltype.l3,aifi_cell_type,cohort.cohortGuid,subject.subjectGuid,subject.birthYear,subject.biologicalSex,sample.visitName,rna_file.id,atac_file.id
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,dc6d9d6831b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.8999748,CD4 TCM,0.3633358,CD4 TCM_3,t_cd4_em,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a
2,dc6f966831b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.8149228,CD4 Naive,0.8149228,CD4 Naive,t_cd4_naive,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a
3,dc8f1c7c31b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.4632712,CD4 TCM,0.3341503,CD4 Naive,t_cd4_naive,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a
4,dca1879a31b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.9009319,CD4 Naive,0.9009319,CD4 Naive,t_cd4_naive,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a
5,dcb6a9f431b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.5730879,CD4 TCM,0.3885105,CD4 TCM_2,t_cd4_cm,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a
6,dcb8a98431b011ef80e742c13d66f8da,KT00395,1,CD4 T,0.8962496,CD4 TCM,0.5954403,CD4 TCM_1,t_cd4_naive,BR2,BR2005,1963,Female,Flu Year 1 Day 7,c9a1c7af-f0c3-41ad-87a7-2a3fb07e682d,32f3d133-f343-4a1e-aca7-13fc3db2f41a


Get counts of each cell type for each sample:

In [11]:
count_summary <- all_labels %>%
  group_by(sample.sampleKitGuid, aifi_cell_type) %>%
  summarise(n_cells = n(),
            .groups = "keep") %>%
  ungroup()

In [18]:
type_minimums <- count_summary %>%
  group_by(aifi_cell_type) %>%
  summarise(n_sample = min(n_cells))

In [19]:
type_minimums %>%
  arrange(aifi_cell_type)

aifi_cell_type,n_sample
<chr>,<int>
t_cd4_cm,1312
t_cd4_em,427
t_cd4_naive,2369
t_cd4_treg,285
t_cd8_memory,402
t_cd8_naive,158


In [20]:
comp_list <- map(
    1:nrow(type_minimums),
    function(i) {
        as.list(type_minimums[i,])
    }
)

## Sample cells for each test

Here, we'll sample cells for comparisons and generate a table of foreground and background cells to use for analysis.

In [23]:
sampled_comp_cells <- map(
    comp_list,
    function(comp) {
        set.seed(3030)

        ct <- comp$aifi_cell_type
        n_sample <- comp$n_sample

        all_labels <- all_labels %>%
          filter(aifi_cell_type == ct)

        sampled_labels <- split(all_labels, all_labels$sample.sampleKitGuid) %>%
          map(sample_n, n_sample) %>%
          list_rbind()

        sampled_labels

    }
)

## Build matrices for each test

Now, we'll use the selected cells to build a data matrix for each comparison.

We'll use these together with the cell metadata to run MAST. Because some steps in the analysis are single-threaded, I've found that it's quite efficient to build a list of datasets and run MAST on each comparison using its own core, rather than rely on the built-in parallelization provided by MAST. This may not be possible on larger datasets, where duplication of data in RAM would be prohibitive (i.e. the shared DMSO control data), but on the scale used for this analysis we should get away with it.

In [32]:
cd4_so <- readRDS(file_paths$cd4_so)
cd8_so <- readRDS(file_paths$cd8_so)

In [33]:
all_so <- merge(cd4_so, cd8_so)
all_so <- JoinLayers(all_so, layers = c("counts"))

In [34]:
all_so <- NormalizeData(
    all_so, 
    normalization.method = "LogNormalize"
)

Normalizing layer: counts



In [35]:
all_mat <- all_so[["RNA"]]@layers$data
rownames(all_mat) <- rownames(all_so)
colnames(all_mat) <- all_so@meta.data$barcodes
rm(cd4_so)
rm(cd8_so)
rm(all_so)

In [36]:
sampled_comp_mats <- map(
    sampled_comp_cells,
    function(meta) {
        all_mat[,meta$barcodes]
    }
)

## Filter genes prior to testing

Before we perform differential tests, we'll remove genes that have low expression in either test group. For our purposes, we'll remove any gene that isn't expressed in at least 5% of either the treatment or DMSO control cells. Note that we don't require that the gene be expressed in 5% of **both** groups - a difference between low/no expression in one group and > 5% expression in another group would still be good to capture.

In [37]:
min_gene_frac <- 0.05

In [38]:
filtered_comp_mats <- map2(
    sampled_comp_cells,
    sampled_comp_mats,
    function(meta, mat) {
        fg_meta <- meta %>%
          filter(cohort.cohortGuid == "BR2")
        bg_meta <- meta %>%
          filter(cohort.cohortGuid == "UP1")

        fg_mat <- mat[,fg_meta$barcodes]
        bg_mat <- mat[,bg_meta$barcodes]

        # Running diff on the pointers of transposed sparse matrices is a pretty fast way to get # non-zero
        fg_fracs <- diff(t(fg_mat)@p) / ncol(fg_mat)
        bg_fracs <- diff(t(bg_mat)@p) / ncol(bg_mat)

        keep_genes <- fg_fracs > min_gene_frac | bg_fracs > min_gene_frac

        mat[keep_genes,]
    }
)

In [39]:
map_int(filtered_comp_mats, nrow)

[1] 7627 7533 6848 7285 7546 7452

## Compute CDR values

To account for technical factors that may affect gene detection in each cell, we'll also need to calculate Cellular Detection Rate values for each cell (CDR), and use these as a cofactor in our MAST equation.

Since we just filtered genes, we'll compute the CDR based on the remaining genes used for analysis for each cell, and add that to our metadata

In [40]:
sampled_comp_cells <- map2(
    sampled_comp_cells,
    filtered_comp_mats,
    function(meta, mat) {
        gene_counts <- diff(mat@p)

        meta %>%
          mutate(cdr = scale(gene_counts))
    }
)

## Run MAST tests

Now, we have all of the pieces we need to run our MAST tests easily. We'll run these across multiple cores using `furrr`'s `future_map2()`.

Because we're mainly interested in the difference between cohorts, we'll include sample.sampleKitGuid in the model formula for MAST to capture sample-specific differences:
`~ cohort.cohortGuid + sample.sampleKitGuid + cdr`

In [41]:
future::plan(future::multisession, workers = 6)

In [42]:
quietly <- function(...) { suppressMessages(suppressWarnings(...)) }

In [43]:
zlm_res_list <- future_map2(
    sampled_comp_cells,
    filtered_comp_mats,
    function(meta, mat) {
        
        cohort_levels <- c("UP1", "BR2")
        meta$cohort.cohortGuid <- factor(meta$cohort.cohortGuid, levels = cohort_levels)
        
        fdat <- data.frame(x = rownames(mat))
        rownames(fdat) <- rownames(mat)
        
        sca <- FromMatrix(
            exprsArray = as.matrix(mat),
            cData = meta,
            fData = fdat)

        zlm_res <- quietly(
            zlm(formula = formula(~ cohort.cohortGuid + sample.sampleKitGuid + cdr), 
                sca = sca,
                method = "bayesglm",
                ebayes = TRUE,
                parallel = FALSE
            )
        )

        zlm_res
    }
)

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.

`fData` has no primerid.  I'll make something up.

`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et i

### Extract results based on cohort

In [45]:
lrt_list <- "cohort.cohortGuidBR2"

mast_res_list <- future_map2(
    zlm_res_list,
    lrt_list,
    function(zlm_res, lrt_vars) {
        suppressMessages(
            MAST::summary(
                object = zlm_res, 
                doLRT = lrt_vars)
        )
    }
)

### Format results

In [46]:
formatted_mast_res <- map2(
    mast_res_list,
    comp_list,
    function(res, comp) {
        all_res <- res$datatable %>%
          as.data.frame() %>%
          filter(contrast == "cohort.cohortGuidBR2")

        split_res <- split(all_res, all_res$component)

        split_res$H %>%
          filter(contrast == "cohort.cohortGuidBR2") %>%
          dplyr::rename(
              gene = primerid,
              nomP = `Pr(>Chisq)`) %>%
          mutate(
              fg = "BR2",
              bg = "UP1",
              aifi_cell_type = comp$aifi_cell_type,
              n_sample = comp$n_sample,
              logFC = split_res$logFC$coef,
              coef_C = split_res$C$coef,
              coef_D = split_res$D$coef) %>%
          select(aifi_cell_type, fg, bg, n_sample,
                 gene, coef_C, coef_D, logFC, nomP) %>%
          mutate(adjP = p.adjust(nomP, method = "BH"))
                 
    }
)

In [47]:
all_res <- do.call(rbind, formatted_mast_res)

In [48]:
head(all_res)

,aifi_cell_type,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP
,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,t_cd4_cm,BR2,UP1,1312,A2M,0.06400018,-0.01892151,0.007134301,0.34500126,0.5584305
2,t_cd4_cm,BR2,UP1,1312,A2M-AS1,-0.07074156,-0.24139603,-0.021597095,0.08162303,0.2308264
3,t_cd4_cm,BR2,UP1,1312,AAGAB,-0.01773918,-0.07900734,-0.015881872,0.71228523,0.8283927
4,t_cd4_cm,BR2,UP1,1312,AAK1,-0.01074648,-0.22092179,-0.050144302,0.23676595,0.4501032
5,t_cd4_cm,BR2,UP1,1312,AAMDC,0.01618775,0.03079666,0.008204034,0.84067210,0.9067750
6,t_cd4_cm,BR2,UP1,1312,AAR2,-0.04863600,-0.45211817,-0.029831443,0.02945882,0.1161347


In [49]:
sig_res <- all_res %>%
  filter(adjP < 0.01)

In [50]:
table(sig_res$aifi_cell_type)


    t_cd4_cm     t_cd4_em  t_cd4_naive   t_cd4_treg t_cd8_memory  t_cd8_naive 
         916          235         1376          105          402           55 

In [52]:
sig_res %>%
  filter(gene %in% c("SOX4", "TSHZ2"))

aifi_cell_type,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
t_cd4_naive,BR2,UP1,2369,SOX4,0.01367318,-1.938523,-0.1756116,3.531259e-42,3.405924e-40
t_cd4_naive,BR2,UP1,2369,TSHZ2,0.23779632,1.523180,1.0966906,1.601622e-149,2.193581e-146


## Generate output files

For downstream use, we'll output the table of aifi_cell_type labels for each cell.

In [53]:
dir.create("output")

Warning message in dir.create("output"):
“'output' already exists”


In [54]:
write.csv(all_res,
          paste0("output/all_mast_deg_",Sys.Date(),".csv"),
          quote = FALSE, row.names = FALSE)

## Store results in HISE

Finally, we store the output file in our Collaboration Space for later retrieval and use. We need to provide the UUID for our Collaboration Space (aka `studySpaceId`), as well as a title for this step in our analysis process.

The hise function `uploadFiles()` also requires the FileIDs from the original fileset for reference.

In [55]:
study_space_uuid <- "00a53fa5-18da-4333-84cb-3cc0b0761201"
title <- paste("TEA-seq demo MAST DEG", Sys.Date())

In [57]:
search_id <- ids::adjective_animal()
search_id

[1] "waspish_leopard"

In [60]:
in_list <- file_uuids
in_list["meta_uuid"] <- meta_uuid

In [61]:
in_list

$cd4_so
[1] "50756d6d-8216-4eba-8245-0a41d4f44e7a"

$cd8_so
[1] "057a9e09-1f9d-4684-b00f-7739209ffe17"

$cd4_labels
[1] "56da30a4-3ac1-4c2b-8ca8-5e1d58cc6986"

$cd8_labels
[1] "d369cedf-18c3-4579-83ea-a0daab116e3c"

$meta_uuid
[1] "5e3115d4-9207-4020-8e3a-3792dd28ea6b"

In [58]:
out_list <- list(paste0("output/all_mast_deg_",Sys.Date(),".csv"))

In [59]:
out_list

[[1]]
[1] "output/all_mast_deg_2024-07-01.csv"

In [65]:
uploadFiles(
    files = out_list,
    studySpaceId = study_space_uuid,
    title = title,
    inputFileIds = file_uuids,
    destination = search_id
)

You are trying to upload the following files:  output/all_mast_deg_2024-07-01.csv



(y/n) y


$files
$files[[1]]
[1] "output/all_mast_deg_2024-07-01.csv"


$traceId
[1] "76aac3a8-6d18-4024-839d-0bf9821b07f1"

In [66]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] MAST_1.28.0                 SingleCellExperiment_1.24.0
 [3] SummarizedExperiment_1.32.0 Biobase_2.62.0             
 [5] GenomicRanges_1.54.1        GenomeInfoDb_1.38.5        
 [7] IRanges_2.36.0              S4Vectors_0.40.2           
 [9] BiocGenerics_0.48.1         MatrixGenerics_1.14.0 